# Reading Oxford Nanopore sequencing data into Python.
Data from nanopore sequencing are stored in a fast5 (HDF) file, which essentially is a container of various object types.  
Before we do any kind of analysis on this data we will need to do some "Data Wrangling" to get it into python, and into the format we want.  
We will first import all of the python modules we are going to need. Of particular note here is **h5py**. The best way to install this is to open a terminal window and type the following (without the $):  
>`$ conda install h5py`  

You may need to close this notebook, and any terminal windows, and reopen jupyter notebook for the import to work.  
If you are on a Windows machine, or you dont have Anaconda, go to http://docs.h5py.org/en/latest/build.html for instructions on how to install this module on your machine.

In [1]:
import h5py as h5, numpy as np, os, sys, urllib, lxml.html

Firstly, we are going to load some example fast5 files into python with some web scraping

In [10]:
path = "http://data.genomicsresearch.org/internal/Nanopore/Ecoli/porecamp/"
connection = urllib.urlopen(path)

The next block will get the URL links for all of the example fast5 files.

In [3]:
dom = lxml.html.fromstring(connection.read())
f5_files = []
for link in dom.xpath('//a/@href')[5:]: # first 5 links are not files on this page
    f5_files.append(link)

In [4]:
# here is the file names for the fast5 files we are going to look at
for filename in f5_files:
    print filename

5CG6210Y8Z_20160816_FNFAB28012_MN15120_mux_scan_GroupB_1D_Ecoli_2016_53944_ch111_read44_strand.fast5
5CG6210Y8Z_20160816_FNFAB28012_MN15120_sequencing_run_GroupB_1D_Ecoli_2016_67932_ch100_read1054_strand.fast5
5CG6210Y8Z_20160816_FNFAB28012_MN15120_sequencing_run_GroupB_1D_Ecoli_2016_67932_ch100_read1103_strand.fast5
5CG6210Y8Z_20160816_FNFAB28012_MN15120_sequencing_run_GroupB_1D_Ecoli_2016_67932_ch100_read1111_strand.fast5
5CG6210Y8Z_20160816_FNFAB28012_MN15120_sequencing_run_GroupB_1D_Ecoli_2016_67932_ch100_read1451_strand.fast5
5CG6210Y8Z_20160816_FNFAB28012_MN15120_sequencing_run_GroupB_1D_Ecoli_2016_67932_ch100_read1731_strand.fast5


In [5]:
# turn these filenames into their respective URLs
f5_urls = [path + f for f in f5_files]

This line will copy these fast5 files into your current directory on your local machine so we can start using them (the files are not very big ~1-2MB).

In [6]:
for i, f in enumerate(f5_urls):
    opener = urllib.URLopener()
    opener.retrieve(f, f5_files[i])

## Get FAST5 files into python and start wrangling
Now we are going to do the extremely import steps of getting our FAST5 files read for our training.  

Lets define a function that, given a filename, will load the FAST5 file in and extract the bits we are interested in for our analysis.

In [7]:
def load_and_extract_fast5(filename):
    """Function reads in a FAST5 file and returns the detected events, 
    the base-called events and the FASTQ file.
    
    load_and_extract_fast5(string) -> np.array, np.string_, np.array
    
    Preconditions: Must have h5py and numpy modules installed. Also,
    this function will only extract 1D information.
    """
    import h5py as h5, numpy as np, os, sys
    # a list to sotre the names of the "groups" - similar to keys in a dictionary
    lst_names = []
    # open the fast5 file - closes automatically when loop is finished.
    with h5.File(filename, 'r') as f:
        # gets the name of all of the groups and stores them in our list
        f.visit(lst_names.append)
        
        # get the names of the groups we are interested in
        groups = []
        for name in lst_names:
            if name.endswith("Events") or name.endswith("template/Fastq"):
                if not name.endswith("complement/Events") and "2D" not in name:
                    groups.append(name)
        
        # catch if there are more than 3 elements in the groups that have been extracted
        if len(groups) != 3:
            raise IndexError("{0} groups detected. Need 3 groups. Speak to Michael if you see this.".format(len(groups)))

        
        # extract the information from the basecalled events
        events = f[groups[0]].value
        called_events = np.array([x for xs in events for x in xs]).reshape(len(events), 
                                                                           len(events[0]))
        
        # extract the FASTQ file
        fastq = f[groups[1]].value
        
        # extract the information from the detected events - before basecalling
        dataset = f[groups[2]].value
        event_detection = np.array([x for xs in dataset for x in xs]).reshape(len(dataset), 
                                                                              len(dataset[0]))
         
        return called_events, fastq, event_detection

Now that we have a function that can extract the information we want, lets go ahead and implement this function on our FAST5 files.  
For the purpose of this example we will store them in a dictionary whose key will be the read id. For future though you may want to use/name/store the data differently. i.e here we set the keys to be the read id, but read ID may not necessarily be a unique identifier in other circumstances. Always check this first.

In [8]:
f5_data = {}
for f5 in f5_files:
    f5_data[f5.split("_")[-2]] = load_and_extract_fast5(f5)

These event files do not have column names. For the sake of understanding we will also generate two lists that correspond to column names for the 2 events arrays.

In [9]:
called_events_colnames = ["mean", "start", "stdv", "length", "model_state", 
                          "move", "weights", "p_model_state", "mp_state", 
                          "p_mp_state", "p_A", "p_C", "p_G", "p_T"]
event_detection_colnames = ["start", "length", "mean", "stdv"]

### Closing Remarks
Bare in mind that the purpose of this notebook was to help you get your head around loading and manipulating data - specifically FAST5 files - in python. Whilst it is intended that some of the above code will help with curating our dataset for the training of our neural network, science does not always play nice and we have to troubleshoot and problem solve to find ways around these obstacles.  
What I mean by this is that **when running this notebook make sure you play around with the code and add in lots of print statements to see what is happening under the hood** - especially in lines that you do not understand.